In [138]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.linalg import lu, solve_triangular, solve

In [139]:
# define model
H1=60
A1=0.05
L1=500

H2=45
A2=0.03
L2=2500

ro=1000
g=9.81
Zeta=0.002
p0=1e6
tol = 10

def differential_pressure_function(x, p_el):
    a = 4/27*p0**3/(p_el-1e5)**2
    b = p0/2
    iv_tot = x[0] + x[1]

    return np.array([
        (p0-a*iv_tot**2-b*iv_tot**3)-(L1 * Zeta * ro / 2 / A1**2 * x[0]**2)-(ro*g*H1), # f1
        (p0-a*iv_tot**2-b*iv_tot**3)-(L2 * Zeta * ro / 2 / A2**2 * x[1]**2)-(ro*g*H2), # f2
    ])
    
def differential_pressure_function_jacobi(x, p_el):
    a = 4/27*p0**3/(p_el-1e5)**2
    b = p0/2
    iv_tot = x[0] + x[1]

    return np.array([
        [
            (-2*a*iv_tot-3*b*iv_tot**2)-(L1 * Zeta * ro / 2 / A1**2 * 2 * x[0]), # f1 nach Iv1
            (-2*a*iv_tot-3*b*iv_tot**2) # f1 nach Iv2
        ],
        [
            (-2*a*iv_tot-3*b*iv_tot**2), # f2 nach Iv1
            (-2*a*iv_tot-3*b*iv_tot**2)-(L2 * Zeta * ro / 2 / A2**2 * 2 * x[1]) # f2 nach Iv2
        ]
    ])

In [140]:
# generalized gauss algorythm
class AttrDict(dict):
    def __getattr__(self, key):
        return self[key]

    def __setattr__(self, key, value):
        self[key] = value

def newton(f, df, x, tolerance=1e-14, max_step=100, debug=False):
    xs = []
    step = 0
    error = np.linalg.norm(f(x))
    while error > tolerance and step < max_step:
        if debug: print(f"Error: {error}, X: {x}, Step: {step}")
        x += solve(df(x), -f(x))
        error = np.linalg.norm(f(x))
        step += 1
        xs.append(x)

    return xs, error, step, x

In [141]:
# get model functions with desired parameters
f = lambda x: differential_pressure_function(x, p_el=200_000)
df = lambda x: differential_pressure_function_jacobi(x, p_el=200_000)

x0 = np.array([0.1, 0.1])
(xs, error, num_steps, x) = newton(f, df, x0)
print(x)

[-0.06480965  0.23081686]
